In [10]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDClassifier

In [11]:
# pickleをデータフレームに復元
df_csv = pd.read_csv("./ks-projects-201801.csv")
df_csv.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


In [12]:
df_csv.count()

ID                  378661
name                378657
category            378661
main_category       378661
currency            378661
deadline            378661
goal                378661
launched            378661
pledged             378661
state               378661
backers             378661
country             378661
usd pledged         374864
usd_pledged_real    378661
usd_goal_real       378661
dtype: int64

In [13]:
#必要でなさそうな変数削除
df_csv.drop('ID', axis=1, inplace=True)
df_csv.drop('name', axis=1, inplace=True)
df_csv.drop('pledged', axis=1, inplace=True)
df_csv.drop('usd pledged', axis=1, inplace=True)
df_csv.drop('usd_pledged_real', axis=1, inplace=True)
df_csv.drop('usd_goal_real', axis=1, inplace=True)
df_csv.head()

,category,main_category,currency,deadline,goal,launched,state,backers,country
0,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,failed,0,GB
1,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,failed,15,US
2,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,failed,3,US
3,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,failed,1,US
4,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,canceled,14,US


In [14]:
#空白値の確認
df_csv.isnull().sum()

category         0
main_category    0
currency         0
deadline         0
goal             0
launched         0
state            0
backers          0
country          0
dtype: int64

In [15]:
#df_csv.describe()

In [16]:
#成功か不成功かを判断したいのでsuccessful,failed以外は除外
df_csv = df_csv[df_csv['state'] != 'undefined']
df_csv = df_csv[df_csv['state'] != 'live']
df_csv = df_csv[df_csv['state'] != 'suspended']
df_csv = df_csv[df_csv['state'] != 'canceled']
df_csv.count()

category         331675
main_category    331675
currency         331675
deadline         331675
goal             331675
launched         331675
state            331675
backers          331675
country          331675
dtype: int64

In [17]:
#ダミー変数処理
df_csv=pd.get_dummies(df_csv, columns=["category", "main_category", "currency","country"], sparse=True)

#ダミー変数化したため、元の列が不必要となり、削除する
#df_csv.drop('category', axis=1, inplace=True)
#df_csv.drop('main_category', axis=1, inplace=True)
#df_csv.drop('currency', axis=1, inplace=True)
#df_csv.drop('country', axis=1, inplace=True)
df_csv.head()


,deadline,goal,launched,state,backers,category_3D Printing,category_Academic,category_Accessories,category_Action,category_Animals,...,country_JP,country_LU,country_MX,"country_N,0""",country_NL,country_NO,country_NZ,country_SE,country_SG,country_US
0,2015-10-09,1000.0,2015-08-11 12:12:28,failed,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2017-11-01,30000.0,2017-09-02 04:43:57,failed,15,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2013-02-26,45000.0,2013-01-12 00:20:50,failed,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,2012-04-16,5000.0,2012-03-17 03:24:11,failed,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,2016-04-01,50000.0,2016-02-26 13:38:27,successful,224,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [18]:
#質的変数を量的変数に変換:決定木モデルならone-hotよりlabelのほうがよい?
#そもそもダミー変数だらけになる
#from sklearn.preprocessing import LabelEncoder

#ラベルを覚えさせる
#le_main = LabelEncoder().fit(df_csv['main_category'])
#le_cat = LabelEncoder().fit(df_csv['category'])
#le_cur = LabelEncoder().fit(df_csv['currency'])
#le_cou = LabelEncoder().fit(df_csv['country'])

#ラベルを整数に変換
#df_csv['main_category'] = le_main.transform(df_csv['main_category'])
#df_csv['category'] = le_cat.transform(df_csv['category'])
#df_csv['currency'] = le_cur.transform(df_csv['currency'])
#df_csv['country'] = le_cou.transform(df_csv['country'])

In [19]:
#目的変数を真理値に変換
df_csv.loc[df_csv['state'] == 'failed', 'state'] = 0
df_csv.loc[df_csv['state'] == 'successful', 'state'] = 1
df_csv.head()

,deadline,goal,launched,state,backers,category_3D Printing,category_Academic,category_Accessories,category_Action,category_Animals,...,country_JP,country_LU,country_MX,"country_N,0""",country_NL,country_NO,country_NZ,country_SE,country_SG,country_US
0,2015-10-09,1000.0,2015-08-11 12:12:28,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2017-11-01,30000.0,2017-09-02 04:43:57,0,15,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2013-02-26,45000.0,2013-01-12 00:20:50,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,2012-04-16,5000.0,2012-03-17 03:24:11,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,2016-04-01,50000.0,2016-02-26 13:38:27,1,224,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [20]:
#日付データをfloatに変換
df_csv['deadline'] = pd.to_datetime(df_csv['deadline'], format='%Y-%m-%d %H:%M:%S')
df_csv['launched'] = pd.to_datetime(df_csv['launched'], format='%Y-%m-%d %H:%M:%S')
df_csv['deadline'] = df_csv['deadline'].map(pd.Timestamp.timestamp)
df_csv['launched'] = df_csv['launched'].map(pd.Timestamp.timestamp)
df_csv.head()

,deadline,goal,launched,state,backers,category_3D Printing,category_Academic,category_Accessories,category_Action,category_Animals,...,country_JP,country_LU,country_MX,"country_N,0""",country_NL,country_NO,country_NZ,country_SE,country_SG,country_US
0,1.444349e+09,1000.0,1.439295e+09,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1.509494e+09,30000.0,1.504327e+09,0,15,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1.361837e+09,45000.0,1.357950e+09,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1.334534e+09,5000.0,1.331955e+09,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,1.459469e+09,50000.0,1.456494e+09,1,224,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [21]:
#開始日と締め切り日の差を新規に作成
#締切日が長いほど、集金額が増えると考えたため
df_csv['span'] = df_csv['deadline'] - df_csv['launched']

In [22]:
#ゴールの額と集金期間で集金のむずかしさが出てくると予測
#replace:変換,inplace:上書きでnp.infの値をnp.nanに上書きしている
#np.infを残しておくと、データが読み取れずエラーになるため0で上書きする

#df_csv['goal_div_span'] = df_csv['goal'] / df_csv['span']
#df_csv.replace([np.inf,-np.inf],np.nan,inplace=True)
#df_csv.fillna(0,inplace=True)

In [23]:
#標準化してから
#df_csv.plot.box(figsize=(15,10))

In [24]:
#散布図：列が多いため、表示されない(ラベルエンコードで確認済み)
#df_csv.corr()
#pd.plotting.scatter_matrix(train, figsize=(14,14))
#plt.show()

In [25]:
#相関係数：列が多いため、表示されない(ラベルエンコードで確認済み)
#sns.heatmap(df_csv.corr(),annot=True,fmt='.2f')

In [26]:
#支援者数と寄付金額に高い相関があるのでどちらかでよさそう
#deadline,launchedはほぼ同じデータ spanは関係ない
#目標額・募集期間に外れ値
df_csv.drop('launched', axis=1, inplace=True)

In [27]:
# ロジスティック回帰
from  sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix 
X = df_csv.drop('state', axis=1).values
y = df_csv["state"].values
clf = SGDClassifier(loss='log', penalty='none', max_iter=10000, fit_intercept=True, random_state=1234, tol=1e-3)
clf.fit(X, y)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=10000,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='none',
       power_t=0.5, random_state=1234, shuffle=True, tol=0.001,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [28]:
# ラベルを予測
y_pred = clf.predict(X)

# 正答率を計算
accuracy =  accuracy_score(y, y_pred)
print('正答率（Accuracy） = {:.3f}%'.format(100 * accuracy))

# Precision, Recall, F1-scoreを計算
precision, recall, f1_score, _ = precision_recall_fscore_support(y, y_pred)

# カテゴリ「2000万以上」に関するPrecision, Recall, F1-scoreを表示
print('適合率（Precision） = {:.3f}%'.format(100 * precision[1]))
print('再現率（Recall） = {:.3f}%'.format(100 * recall[1]))

正答率（Accuracy） = 59.818%
適合率（Precision） = 58.117%
再現率（Recall） = 1.820%


In [29]:
#クロス集計
conf_mat = pd.DataFrame(confusion_matrix(y, y_pred), 
                        index=['正解 : 失敗', '正解 : 成功'], 
                        columns=['予測 : 失敗', '予測 : 成功'])
conf_mat

,予測 : 失敗,予測 : 成功
正解 : 失敗,195962,1757
正解 : 成功,131518,2438
